# Введение в глубинное обучение, ФКН ВШЭ

# Практическое задание 2. Рекуррентные Нейронные Сети

## Общая информация
Дата выдачи: 04.02.2021

Мягкий дедлайн: 19.02.2021 00:59 MSK

Жёсткий дедлайн: 22.02.2021 00:59 MSK

## Оценивание и штрафы

Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов.

Сдавать задание после указанного срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).


## Формат сдачи
Задания сдаются семинаристу на почту. Посылка должна содержать:
* Ноутбук homework-practice-02-Username.ipynb

Username — ваша фамилия на латинице

## 0. Подготовка данных (0 баллов)

Данные представляют собой корпус текстов с 4-мя категориями. Ваша задача - написать классификатор для этих данных, определяющий, к какой из категорий относится текст.

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import LSTM, RNN, LSTMCell, Activation, Dense, Dropout,Input, Embedding,SpatialDropout1D,Bidirectional,Conv1D, GlobalMaxPooling1D, BatchNormalization,Flatten
from keras.optimizers import RMSprop
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping
#import torch
#from torchtext import data
#import torch.nn as nn
%matplotlib inline

df_train = pd.read_csv('./train.csv')


In [62]:
import re
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()
def lemmatize(doc):
    doc = re.sub(patterns, ' ', str(doc))
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            
            tokens.append(token)

    return tokens

In [63]:
df_train['text'] = df_train['text'].apply(lemmatize)

In [64]:
#df_train = df_train[df_train['text'] != None]
list_none = []
for i,row in df_train.iterrows():
    if row['text'] == None:
        list_none.append(i)
list_none

[]

In [65]:
#df_train.drop(df_train.index[list_none], inplace = True)


## 1. Предобработка данных (2 балла)

В этом задании вам предстоит провести предобработку данных. Баллы ставятся следующим образом:

* Привести все тексты к одной длине, заменить слова/токены на числа, факторизовать целевую переменную и т.д. (1 балл)

* Использовать токенизатор, который разбил бы все слова на токены (подробнее https://github.com/huggingface/tokenizers). (1 балл)

In [66]:
y = df_train['source']
X = df_train['text']

In [67]:
train_text,test_text,train_cat,test_cat = train_test_split(X,y,test_size=0.15)


In [68]:
max_features = 5000
max_len = 250
tok = Tokenizer(num_words=max_features)
tok.fit_on_texts(X)
sequences = tok.texts_to_sequences(train_text)
X_train = sequence.pad_sequences(sequences,maxlen=max_len,padding="post")

test_sequences = tok.texts_to_sequences(test_text)
X_test = sequence.pad_sequences(test_sequences,maxlen=max_len,padding="post")


In [69]:
encoder = LabelEncoder()
encoder.fit(train_cat)
y_train = encoder.transform(train_cat)
y_test = encoder.transform(test_cat)
y_train = to_categorical(y_train, 4)
y_test = to_categorical(y_test, 4)

## 2. LSTM-сеть (4 балла)

В этом задании вам предстоит написать LSTM сеть __вручную__ (то есть без использования стандартных реализаций из keras / torch / tensorflow). Сама архитектура отлично расписана здесь: https://colah.github.io/posts/2015-08-Understanding-LSTMs/

Пытался реализовать несколько дней,облазил все статьи на towards, git и др.  но не нашел как это встроить в модели для торча )));

## 3. Модель (2 балла)

В этом задании вам предстоит объединить вашу сеть с несколькими другими слоями для создания итоговой модели классификатора (можно начать с самой базовой архитектуры, слой эмбеддингов - LSTM - выходной слой)

In [70]:
model = Sequential()
model.add(Embedding(max_features,256, input_length = X_train.shape[1]))
model.add(Conv1D(64, 7, padding="valid",activation="relu",strides=3))
model.add(SpatialDropout1D(0.2))

#model.add(Conv1D(64, 7, padding="valid",activation="relu",strides=5))
model.add(Conv1D(128, 7, padding="valid",activation="relu",strides=3))
#model.add(Flatten())
model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
#model.add(GlobalMaxPooling1D())
#model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))

model.add(Flatten())

model.add(Dense(4,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 250, 256)          1280000   
_________________________________________________________________
conv1d_28 (Conv1D)           (None, 82, 64)            114752    
_________________________________________________________________
spatial_dropout1d_20 (Spatia (None, 82, 64)            0         
_________________________________________________________________
conv1d_29 (Conv1D)           (None, 26, 128)           57472     
_________________________________________________________________
bidirectional_20 (Bidirectio (None, 256)               263168    
_________________________________________________________________
flatten_15 (Flatten)         (None, 256)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 4)               

In [71]:
batch_size = 64
model.fit(X_train, y_train, epochs=3 , verbose=1, batch_size = batch_size, validation_data=(X_test,y_test) )

Epoch 1/3
132/132 [==============================] - 91s 536ms/step - loss: 0.4261 - accuracy: 0.5497 - val_loss: 0.0675 - val_accuracy: 0.9663
Epoch 2/3
132/132 [==============================] - 67s 507ms/step - loss: 0.0619 - accuracy: 0.9689 - val_loss: 0.0581 - val_accuracy: 0.9697
Epoch 3/3
132/132 [==============================] - 67s 510ms/step - loss: 0.0520 - accuracy: 0.9723 - val_loss: 0.0580 - val_accuracy: 0.9717


In [72]:
model.evaluate(X_test, y_test)

47/47 [==============================] - 4s 76ms/step - loss: 0.0580 - accuracy: 0.9717


[0.0579695850610733, 0.9716790318489075]

In [ ]:
test = pd.read_csv('test.csv')

predicts = model.predict(test['text'])

## 4. Обучение модели (4 балла)

**Важно!** Public Leaderboard содержит только 30% тестовых данных, баллы за задание будут измеряться в соответствии со всеми тестовыми данными, так что после завершения соревнования Ваша позиция на leaderboard-е может поменяться.

**Важно!** При использовании **СВОЕЙ** модели LSTM полученные баллы удваиваются.

* val_accuracy > baseline: 1 балл

* Ваша позиция на лидерборде между 25% (включительно) и 50% (невключительно) квантилями среди студентов, перебивших baseline: 2 балла

* Ваша позиция на лидерборде между 50% и 75% квантилями среди студентов, перебивших baseline: 3 балла

* Ваша позиция на лидерборде между 75% и 90% квантилями среди студентов, перебивших baseline: 4 балла

* Ваша позиция на лидерборде больше 90% квантиля среди студентов, перебивших baseline: 5 баллов

* Вы в топ-3 студентов на лидерборде: 10 баллов


**Максимальный суммарный балл** - 15

**Финальная оценка**: min(суммарный балл, 10), доп баллы можно использовать для увеличения оценок за самостоятельные работы (1 доп балл = +1 балл к самостоятельной)
